In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
library(beaveR)
clustFile <- "../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/no_threshold0/cluster_nwk.txt"
quantDir <- file.path("../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/out_sal") 
samples <- as.vector(outer(c(1:6), c(1,2), function(x,y) paste(x,y,sep='_')))
quantFiles <- file.path(quantDir, samples, 'quant.sf')
coldata <- data.frame(files=quantFiles, names=samples, condition=factor(rep(1:2, each=6)))
tse <- buildTSE(treeTermFile = clustFile, coldata = coldata)

In [4]:
devtools::install_github("NPSDC/beaveR", ref = "dev", force=T)

rlang    (1.0.4        -> 1.0.6       ) [CRAN]
igraph   (1.3.1        -> 1.3.5       ) [CRAN]
digest   (0.6.30       -> 0.6.31      ) [CRAN]
fishpond (bd0f06193... -> 2c057253d...) [GitHub]


Skipping 5 packages not available: S4Vectors, tximeta, TreeSummarizedExperiment, SingleCellExperiment, SummarizedExperiment

Installing 3 packages: rlang, igraph, digest

Warning message in i.p(...):
“installation of package ‘rlang’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘digest’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘igraph’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done




RCurl     (1.98-1.8 -> 1.98-1.9) [CRAN]
vctrs     (0.4.2    -> 0.5.1   ) [CRAN]
rlang     (1.0.4    -> 1.0.6   ) [CRAN]
lifecycle (1.0.1    -> 1.0.3   ) [CRAN]
stringr   (1.4.1    -> 1.5.0   ) [CRAN]
tibble    (3.1.7    -> 3.1.8   ) [CRAN]
ggplot2   (3.3.6    -> 3.4.0   ) [CRAN]
jsonlite  (1.8.3    -> 1.8.4   ) [CRAN]


Installing 8 packages: RCurl, vctrs, rlang, lifecycle, stringr, tibble, ggplot2, jsonlite

Warning message in i.p(...):
“installation of package ‘RCurl’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘rlang’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘jsonlite’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘lifecycle’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘vctrs’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘stringr’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘tibble’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘ggplot2’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



✔  checking for file ‘/cbcbhomes/npsingh/.tmp/RtmpCL7GxB/remotesa8fb37e64497/mikelove-fishpond-2c05725/DESCRIPTION’
─  preparing ‘fishpond’: (567ms)
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts (336ms)
─  checking for empty or unneeded directories
   Omitted ‘LazyData’ from DESCRIPTION
─  building ‘fishpond_2.5.1.tar.gz’
   


Warning message in i.p(...):
“installation of package ‘/cbcbhomes/npsingh/.tmp/RtmpCL7GxB/filea8fb1da6d71d/fishpond_2.5.1.tar.gz’ had non-zero exit status”


✔  checking for file ‘/cbcbhomes/npsingh/.tmp/RtmpCL7GxB/remotesa8fb2438b677/NPSDC-beaveR-2679aba/DESCRIPTION’ (409ms)
─  preparing ‘beaveR’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘beaveR_0.99.0.tar.gz’
   


In [5]:
aggAssay <- function(tree, nodeIDs, seMat, groupInds = NULL) {
    if (!is(tree, "phylo"))
        stop("tree does not belong to class phylo")
    if (!is.numeric(nodeIDs)) {
        nodeIDs <- suppressWarnings(as.numeric(nodeIDs))
        if (any(is.na(nodeIDs)))
            stop("Node ids contain a non numeric")
    }
    if (any(nodeIDs > (length(tree$tip) + tree$Nnode)))
        stop("some nodeIDs are invalid")
    performMatCheck(seMat)
    mat <- createMat(seMat, list(), type = "row")

    leaves <- which(nodeIDs <= nrow(seMat))
    innNodesInds <- which(nodeIDs > nrow(seMat))

    if (length(leaves) > 0) {
        mat <- rbind(mat, seMat[nodeIDs[leaves], ])
        rownames(mat) <- rownames(seMat)[nodeIDs[leaves]]
    }

    if (length(innNodesInds) > 0) {
        lInds <- phangorn::Descendants(tree, nodeIDs[innNodesInds], type = "tips")
        names(lInds) <- paste("Node", as.character(nodeIDs[innNodesInds]), sep = "")
        mat <- rbind(mat, performRowSumAgg(seMat, lInds))
    }
    mat <- performColMeanAgg(mat, groupInds)
    mat
}

#' @importFrom methods is
performColMeanAgg <- function(seM, colInds = NULL) {
    performMatCheck(seM)
    if (is.null(colInds)) {
        return(seM)
    }
    performIndCheck(seM, colInds, type = "col")
    mat <- createMat(seM, colInds, type = "col")
    FUN <- rowMeans
    if (is(seM, "dgCMatrix")) {
        FUN <- Matrix::rowMeans
    }
    for (i in seq_along(colInds)) {
        if (length(colInds[[i]]) != 1) {
            mat[, i] <- FUN(seM[, colInds[[i]]])
        } else {
            mat[, i] <- seM[, colInds[[i]]]
        }
    }
    dimnames(mat) <- getDimNames(seM, colInds, type = "col")
    mat
}

performRowSumAgg <- function(seM, rowInds = NULL) {
    performMatCheck(seM)
    if (is.null(rowInds)) {
        return(seM)
    }
    performIndCheck(seM, rowInds, type = "row")
    mat <- createMat(seM, rowInds, type = "row")
    FUN <- colSums
    if (is(seM, "dgCMatrix")) {
        FUN <- Matrix::colSums
    }
    for (i in seq_along(rowInds)) {
        if (length(rowInds[[i]]) != 1) {
            mat[i, ] <- FUN(seM[rowInds[[i]], ])
        } else {
            mat[i, ] <- seM[rowInds[[i]], ]
        }
    }
    dimnames(mat) <- getDimNames(seM, rowInds, type = "row")
    mat
}

#' @importFrom methods is
performMatCheck <- function(seM) {
    if (!(is(seM, "matrix") | is(seM, "dgCMatrix"))) {
        stop("input matrix should be of class dgCMatrix or matrix")
    }
    if (is(seM, "matrix")) {
        if (!is.numeric(seM)) {
            stop("input matrix should be numeric")
        }
    }
}

#' @importFrom methods is
performIndCheck <- function(seM, inds, type = "row") {
    if (!is(inds, "list")) {
        stop("colInds has to be list")
    }
    if (type == "row") {
        if (any(unlist(inds) > nrow(seM))) {
            stop("Atleast 1 index in rowInds is greater than the total number of rows")
        }
    } else {
        if (any(unlist(inds) > ncol(seM))) {
            stop("Atleast 1 index in colInds is greater than the total number of columns")
        }
    }

}

getDimNames <- function(seM, inds, type = "row") {
    d2name <- names(inds)
    if (is.null(names(inds))) {
        d2name <- paste("Group", seq_along(inds), sep = "")
    }
    dnames <- list(d2name, colnames(seM))
    if (type != "row")
        dnames <- list(rownames(seM), d2name)
    dnames
}

#' @importFrom methods is
createMat <- function(seM, inds, type = "row") {
    nrows <- ifelse(type == "row", length(inds), nrow(seM))
    ncols <- ifelse(type == "col", length(inds), ncol(seM))

    mat <- matrix(0, nrow = nrows, ncol = ncols)
    if (is(seM, "dgCMatrix")) {
        mat <- Matrix::Matrix(0, nrow = nrows, ncol = ncols, sparse = T)
    }
    mat
}


In [7]:
suppressPackageStartupMessages(library(TreeSummarizedExperiment))
suppressPackageStartupMessages(library(SummarizedExperiment))
suppressPackageStartupMessages(library(tximport))
suppressPackageStartupMessages(library(phangorn))

In [39]:
tree <- rowTree(tse)
ntxps <- length(tree$tip.label)
desc <- phangorn::Descendants(tree, 1:nrow(tse))
# txDf <- data.frame(tx = rownames(tse)[unlist(desc)],
#                        groups = rep(paste("Node", seq(length(desc))+1, sep=""), sapply(desc, length)))
innNodes <- ntxps + 1:tree$Nnode

In [61]:
weightedLength <- aggAssay(tree, c(1:ntxps, innNodes), assays(tse)[["abundance"]][1:ntxps,] * assays(tse)[["length"]][1:ntxps,], groupInds = NULL)

In [68]:
assays

new("standardGeneric", .Data = function (x, withDimnames = TRUE, 
    ...) 
standardGeneric("assays"), generic = "assays", package = "SummarizedExperiment", 
    group = list(), valueClass = character(0), signature = "x", 
    default = NULL, skeleton = (function (x, withDimnames = TRUE, 
        ...) 
    stop("invalid call in method dispatch to 'assays' (no default method)", 
        domain = NA))(x, withDimnames, ...))
<bytecode: 0x557255e2ca80>
<environment: 0x557255e33820>
attr(,"generic")
[1] "assays"
attr(,"generic")attr(,"package")
[1] "SummarizedExperiment"
attr(,"package")
[1] "SummarizedExperiment"
attr(,"group")
list()
attr(,"valueClass")
character(0)
attr(,"signature")
[1] "x"
attr(,"default")
`\001NULL\001`
attr(,"skeleton")
(function (x, withDimnames = TRUE, ...) 
stop("invalid call in method dispatch to 'assays' (no default method)", 
    domain = NA))(x, withDimnames, ...)
attr(,"class")
[1] "standardGeneric"
attr(,"class")attr(,"package")
[1] "methods"

In [65]:
lengthMat <- weightedLength / assays(tse)[["abundance"]]
tail(lengthMat)
tail(assays(tse)[["length"]])

,1_1,2_1,3_1,4_1,5_1,6_1,1_2,2_2,3_2,4_2,5_2,6_2
Node254390,6693.633,6242.552,6265.287,6569.219,6198.635,6088.444,6874.087,6504.084,5968.100,6402.313,5967.446,6576.715
Node254391,6693.633,6242.552,6265.287,6569.219,6198.635,6088.444,6874.087,6504.084,5968.100,6402.313,5967.446,6576.715
Node254392,6880.322,6287.013,6286.709,6629.085,6400.754,6125.581,6919.126,6565.311,6141.039,6463.191,6026.083,6813.699
Node254393,6880.322,6287.013,6286.709,6629.085,6400.754,6125.581,6919.126,6565.311,6141.039,6463.191,6026.083,6813.699
Node254394,6808.979,6179.682,6188.297,6590.828,6344.191,6049.349,6875.091,6502.895,6067.875,6423.958,5955.415,6745.714
Node254395,7417.328,7405.501,7406.089,7465.426,7347.743,7290.692,7423.373,7403.641,7423.288,7462.822,7364.343,7294.859


,1_1,2_1,3_1,4_1,5_1,6_1,1_2,2_2,3_2,4_2,5_2,6_2
Node254390,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Node254391,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Node254392,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Node254393,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Node254394,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Node254395,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [40]:
aveLengthSampGroup <- tapply(aveLengthSamp, desc, mean)

ERROR: Error in tapply(aveLengthSamp, desc, mean): arguments must have same length


In [59]:
head(lengthMat)


In [56]:
tail(aveLengthSampGroup)[4]==mean(aveLengthSamp[tail(desc)[[4]]])

[1] TRUE

In [11]:
txDf <- data.frame(tx = rownames(tse)[unlist(desc)], gene = rep(NA, length(unlist(desc))))
j <- 1
for(i in seq_along(desc)) {
    ll <- length(desc[[i]])
    txDf[j:(j+ll-1),2]=paste("Group", i, sep="_")
    j <- j + ll
}

In [39]:
groupCount <- summarizeToGene(tse, tx2gene = txDf, ignoreTxVersion = TRUE)

loading existing TxDb created: 2022-08-19 22:12:59

obtaining transcript-to-gene mapping from database

loading existing gene ranges created: 2022-08-26 02:08:58



ERROR: Error in .local(object, ...): formal argument "tx2gene" matched by multiple actual arguments


In [4]:
txi <- tximeta::tximeta(coldata$files, type = "salmon", countsFromAbundance="lengthScaledTPM")

importing quantifications

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 


found matching transcriptome:
[ GENCODE - Homo sapiens - release 26 ]

loading existing TxDb created: 2022-08-19 22:12:59

Loading required package: GenomicFeatures

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: 'S4Vectors'


The following ob

In [41]:
gse <- tximeta::summarizeToGene(txi, countsFromAbundance="lengthScaledTPM")
gse@metadata$countsFromAbundance

loading existing TxDb created: 2022-08-19 22:12:59

obtaining transcript-to-gene mapping from database

loading existing gene ranges created: 2022-08-26 02:08:58

summarizing abundance

summarizing counts

summarizing length

summarizing inferential replicates



[1] "lengthScaledTPM"

In [43]:
rDf <- rowData(txi)
ids <- which(unlist(rDf$gene_id)=="ENSG00000000419.12")
# table(unlist(rDf$gene_id))
rDf[ids, ]
colSums(assays(txi)[["counts"]][ids,])
colSums(assays(txi)[["infRep1"]][ids,])
assays(txi)[["length"]][ids,]

DataFrame with 6 rows and 3 columns
                      tx_id            gene_id           tx_name
                  <integer>    <CharacterList>       <character>
ENST00000371588.9    184942 ENSG00000000419.12 ENST00000371588.9
ENST00000466152.5    184943 ENSG00000000419.12 ENST00000466152.5
ENST00000371582.8    184944 ENSG00000000419.12 ENST00000371582.8
ENST00000494752.1    184945 ENSG00000000419.12 ENST00000494752.1
ENST00000371584.8    184946 ENSG00000000419.12 ENST00000371584.8
ENST00000413082.1    184947 ENSG00000000419.12 ENST00000413082.1

1        2        3        4        5        6        7        8 
347.0042 359.5480 509.2938 422.1486 477.3536 377.6048 280.9477 482.2791 
       9       10       11       12 
425.7294 512.3368 460.2999 431.4532

1        2        3        4        5        6        7        8 
333.1740 394.1343 497.4392 405.4387 486.3309 371.4274 256.0871 505.0213 
       9       10       11       12 
416.2160 522.7926 413.6535 458.0140

,1,2,3,4,5,6,7,8,9,10,11,12
ENST00000371588.9,810.605,823.192,828.929,798.110,857.396,844.392,811.468,822.563,828.853,800.025,856.969,845.507
ENST00000466152.5,830.850,843.832,849.883,817.910,879.364,865.996,831.749,843.222,849.781,819.866,878.974,867.145
ENST00000371582.8,900.568,910.844,915.574,890.470,938.476,928.410,902.076,911.293,915.835,891.752,938.583,929.721
ENST00000494752.1,535.976,588.000,588.000,588.000,588.000,588.000,588.000,588.000,588.000,588.000,588.000,588.000
ENST00000371584.8,840.123,847.041,849.508,832.057,864.827,862.387,841.396,847.268,849.375,832.737,864.774,863.251
ENST00000413082.1,471.000,471.000,471.000,471.000,471.000,471.000,471.000,471.000,471.000,473.613,471.000,471.000


In [45]:
# txi
# assays(txi)[["infRep1"]]
assays(gse)[["counts"]][3,]
assays(gse)[["infRep1"]][3,]

1        2        3        4        5        6        7        8 
347.1970 360.2469 509.2295 418.7748 477.7686 379.4672 280.4395 483.3569 
       9       10       11       12 
426.8414 511.2760 460.3502 429.9228

1        2        3        4        5        6        7        8 
333.1740 394.1343 497.4392 405.4387 486.3309 371.4274 256.0871 505.0213 
       9       10       11       12 
416.2160 522.7926 413.6535 458.0140

In [48]:
dim(txi[["counts"]])
length(unique(txDf$gene))

[1] 14609    12

[1] 116194

In [35]:
summarizeToGene.list <- function(object,
                                 tx2gene,
                                 varReduce=FALSE,
                                 ignoreTxVersion=FALSE,
                                 ignoreAfterBar=FALSE,
                                 countsFromAbundance=c("no","scaledTPM","lengthScaledTPM")
                                 ) {

  countsFromAbundance <- match.arg(countsFromAbundance, c("no","scaledTPM","lengthScaledTPM"))

  if (!is.null(object$countsFromAbundance)) {
    if (countsFromAbundance == "no" & object$countsFromAbundance != "no") {
      warning(paste0("the incoming counts have countsFromAbundance = '",
                     object$countsFromAbundance,"',
  and so the original counts are no longer accessible.
  to use countsFromAbundance='no', re-run objectmport() with this setting.
  over-riding 'countsFromAbundance' to set it to: ",
  object$countsFromAbundance))
      countsFromAbundance <- object$countsFromAbundance
    }
  }
  
  # unpack matrices from list for cleaner code
  abundanceMatTx <- object$abundance
  countsMatTx <- object$counts
  lengthMatTx <- object$length
  
  txId <- rownames(abundanceMatTx)
  stopifnot(all(txId == rownames(countsMatTx)))
  stopifnot(all(txId == rownames(lengthMatTx)))
  
  # need to associate tx to genes
  # potentially remove unassociated transcript rows and warn user
  if (!is.null(tx2gene)) {

    # code to strip dots or bars and all remaining chars from the rownames of matrices
    if (ignoreTxVersion) {
      txId <- sub("\\..*", "", txId)
    } else if (ignoreAfterBar) {
      txId <- sub("\\|.*", "", txId)
    }
    
    tx2gene <- cleanTx2Gene(tx2gene)    
    
    # if none of the rownames of the matrices (txId) are
    # in the tx2gene table something is wrong
    if (!any(txId %in% tx2gene$tx)) {
      txFromFile <- paste0("Example IDs (file): [", paste(head(txId,3),collapse=", "),", ...]")
      txFromTable <- paste0("Example IDs (tx2gene): [", paste(head(tx2gene$tx,3),collapse=", "),", ...]")
      stop(paste0("
  None of the transcripts in the quantification files are present
  in the first column of tx2gene. Check to see that you are using
  the same annotation for both.\n\n",txFromFile,"\n\n",txFromTable,
  "\n\n  This can sometimes (not always) be fixed using 'ignoreTxVersion' or 'ignoreAfterBar'.\n\n"))
    }

    # remove transcripts (and genes) not in the rownames of matrices
    tx2gene <- tx2gene[tx2gene$tx %in% txId,]
    tx2gene$gene <- droplevels(tx2gene$gene)
    ntxmissing <- sum(!txId %in% tx2gene$tx)
    if (ntxmissing > 0) message("transcripts missing from tx2gene: ", ntxmissing)

    # subset to transcripts in the tx2gene table
    sub.idx <- txId %in% tx2gene$tx
    abundanceMatTx <- abundanceMatTx[sub.idx,,drop=FALSE]
    countsMatTx <- countsMatTx[sub.idx,,drop=FALSE]
    lengthMatTx <- lengthMatTx[sub.idx,,drop=FALSE]
    txId <- txId[sub.idx]

    # now create a vector of geneId which aligns to the matrices
    geneId <- tx2gene$gene[match(txId, tx2gene$tx)]
  }
  
  # summarize abundance and counts
  message("summarizing abundance")
  abundanceMat <- rowsum(abundanceMatTx, geneId)
  message("summarizing counts")
  countsMat <- rowsum(countsMatTx, geneId)
  message("summarizing length")

  if ("infReps" %in% names(object)) {
    infReps <- lapply(object$infReps, function(x) rowsum(x[sub.idx,,drop=FALSE], geneId))
    message("summarizing inferential replicates")
  }
  
  # the next lines calculate a weighted average of transcript length, 
  # weighting by transcript abundance.
  # this can be used as an offset / normalization factor which removes length bias
  # for the differential analysis of estimated counts summarized at the gene level.
  weightedLength <- rowsum(abundanceMatTx * lengthMatTx, geneId)
  lengthMat <- weightedLength / abundanceMat   

  # pre-calculate a simple average transcript length
  # for the case the abundances are all zero for all samples.
  # first, average the tx lengths over samples
  aveLengthSamp <- rowMeans(lengthMatTx)
  # then simple average of lengths within genes (not weighted by abundance)
  aveLengthSampGene <- tapply(aveLengthSamp, geneId, mean)

  stopifnot(all(names(aveLengthSampGene) == rownames(lengthMat)))
  
  # check for NaN and if possible replace these values with geometric mean of other samples.
  # (the geometic mean here implies an offset of 0 on the log scale)
  # NaN come from samples which have abundance of 0 for all isoforms of a gene, and 
  # so we cannot calculate the weighted average. our best guess is to use the average
  # transcript length from the other samples.
  lengthMat <- replaceMissingLength(lengthMat, aveLengthSampGene)

  if (countsFromAbundance != "no") {
    countsMat <- makeCountsFromAbundance(countsMat,
                                         abundanceMat,
                                         lengthMat,
                                         countsFromAbundance)
  }


  if ("infReps" %in% names(object)) {
    if (varReduce) {
      vars <- sapply(infReps, rowVars)
      out <- list(abundance=abundanceMat,
                  counts=countsMat, variance=vars,
                  length=lengthMat,
                  countsFromAbundance=countsFromAbundance)
    } else {
      out <- list(abundance=abundanceMat,
                  counts=countsMat, infReps=infReps,
                  length=lengthMat,
                  countsFromAbundance=countsFromAbundance)
    }
  } else {
    out <- list(abundance=abundanceMat,
                counts=countsMat,
                length=lengthMat,
                countsFromAbundance=countsFromAbundance)
  }
  
  return(out)
}

                      cleanTx2Gene <- function(tx2gene) {
  colnames(tx2gene) <- c("tx","gene")
  if (any(duplicated(tx2gene$tx))) {
    message("removing duplicated transcript rows from tx2gene")
    tx2gene <- tx2gene[!duplicated(tx2gene$tx),]
  }
  tx2gene$gene <- factor(tx2gene$gene)
  tx2gene$tx <- factor(tx2gene$tx)
  tx2gene
}

cleanTx2Gene <- function(tx2gene) {
  colnames(tx2gene) <- c("tx","gene")
  if (any(duplicated(tx2gene$tx))) {
    message("removing duplicated transcript rows from tx2gene")
    tx2gene <- tx2gene[!duplicated(tx2gene$tx),]
  }
  tx2gene$gene <- factor(tx2gene$gene)
  tx2gene$tx <- factor(tx2gene$tx)
  tx2gene
}


In [1]:
x <- matrix(runif(100), ncol = 5)
group <- sample(1:8, 20, TRUE)
(xsum <- rowsum(x, group))


1,0.87552600,0.9778310,0.4960399,0.1004989,0.4016248
3,1.36963795,1.2628045,1.9822154,1.1277403,1.4480252
4,0.61070889,0.2080311,1.3444102,0.8371511,0.9096345
5,1.30829859,2.1519975,1.4206122,0.5605291,2.1411940
6,4.04359137,2.3784320,2.4863981,3.1901702,3.3124241
7,1.40315625,1.3764747,0.9432995,0.9401372,1.5625952
8,0.06906135,1.4107848,0.8286996,0.7271932,1.7707161


In [15]:
which(group==5)

[1]  1 14 19 20

In [16]:
colSums(x[c(1,14,19,20),])

[1] 1.3082986 2.1519975 1.4206122 0.5605291 2.1411940

In [12]:
rowsum(x, group)

1,0.87552600,0.9778310,0.4960399,0.1004989,0.4016248
3,1.36963795,1.2628045,1.9822154,1.1277403,1.4480252
4,0.61070889,0.2080311,1.3444102,0.8371511,0.9096345
5,1.30829859,2.1519975,1.4206122,0.5605291,2.1411940
6,4.04359137,2.3784320,2.4863981,3.1901702,3.3124241
7,1.40315625,1.3764747,0.9432995,0.9401372,1.5625952
8,0.06906135,1.4107848,0.8286996,0.7271932,1.7707161
